In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

(ds_train_raw, ds_test_raw), ds_info = tfds.load(
    'coco/2017',
    split=['train[:1%]', 'validation[:1%]'],
    with_info=True,
    shuffle_files=True
)

In [ ]:
print(f"Number of training images: {ds_train_raw.cardinality().numpy()}")
print(f"Number of testing images: {ds_test_raw.cardinality().numpy()}")

In [ ]:
IMG_SIZE = 128
NUM_CLASSES = ds_info.features['objects']['label'].num_classes

def preprocess(features):
    image = tf.image.resize(features['image'], (IMG_SIZE, IMG_SIZE))
    label = features['objects']['label'][0]
    return image, label

ds_train = ds_train_raw.map(preprocess).cache().shuffle(100).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test_raw.map(preprocess).cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
get_label_name = ds_info.features['objects']['label'].int2str

plt.figure(figsize=(12, 8))
for images, labels in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(get_label_name(labels[i]))
        plt.axis("off")
plt.show()

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

ds_train_augmented = ds_train.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
plt.figure(figsize=(12, 8))
for images, labels in ds_train.take(1):
    augmented_images = data_augmentation(images)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[i].numpy().astype("uint8"))
        plt.title(get_label_name(labels[i]))
        plt.axis("off")
plt.show()

In [ ]:
rescale = tf.keras.layers.Rescaling(1./255)
cnn_model = tf.keras.Sequential([
    rescale,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES)
])

cnn_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history_cnn = cnn_model.fit(
    ds_train_augmented,
    validation_data=ds_test,
    epochs=15
)
loss_cnn, accuracy_cnn = cnn_model.evaluate(ds_test)
print(f"CNN Testing Accuracy: {accuracy_cnn:.2%}")

In [ ]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

transfer_model = tf.keras.Sequential([
    rescale,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES)
])

transfer_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history_transfer = transfer_model.fit(
    ds_train_augmented,
    validation_data=ds_test,
    epochs=15
)
loss_transfer, accuracy_transfer = transfer_model.evaluate(ds_test)
print(f"Transfer Learning (ResNet50) Testing Accuracy: {accuracy_transfer:.2%}")

In [ ]:
cnn_model_no_aug = tf.keras.models.clone_model(cnn_model)
cnn_model_no_aug.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history_no_aug = cnn_model_no_aug.fit(
    ds_train,
    validation_data=ds_test,
    epochs=15
)

loss_no_aug, accuracy_no_aug = cnn_model_no_aug.evaluate(ds_test)

In [ ]:
print("--- Accuracy Comparison ---")
print(f"Simple CNN (No Augmentation): {accuracy_no_aug:.2%}")
print(f"Simple CNN (With Augmentation): {accuracy_cnn:.2%}")
print(f"Transfer Learning (With Augmentation): {accuracy_transfer:.2%}")